# Data Engineering Capstone Project

## Project Summary
For this capstone project, I'm going to do the Udacity given project and data, to analysis the immigrant trend in US in 2016, using AWS EMR and Apache Spark as Data lake. The goal is to understand the statistics of foreign student and workers (F/H Visa), and check which state is most/least popular/friendly to them. And other ad-hoc queries, eg, which state has more foreigners, airports? 

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from pprint import pprint
import configparser
import os
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F 
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date
from collections import OrderedDict 
from datetime import datetime, timedelta

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s  [%(name)s] %(message)s')
LOG = logging.getLogger('etl')

# Step 1: Scope the Project and Gather Data

## Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>
Using the given dataset: i94 in 2016, airport-codes, GlobalLandTemperaturesByCity, us-cities-demographics, I94_SAS_Labels_Descriptions.
Peek into the dataset if we need to pull more external datasets from network, so as to answer our questions/queries.

## Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 
1. The 2016-I94 dataset (SAS7BDAT) is from [the US National Tourism and Trade Office website](https://travel.trade.gov/research/reports/i94/historical/2016.html).
2. World Temperature Data: This dataset came from Kaggle. You can read more about it [here](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).
3. U.S. City Demographic Data: This data comes from OpenSoft. You can read more about it [here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/).
4. Airport Code Table: This is a simple table of airport codes and corresponding cities. It comes from [here](https://datahub.io/core/airport-codes#data).

In [2]:
# Read in the data here
imm_sample = pd.read_csv('./immigration_data_sample.csv')
imm_sample.head()
imm_header = list(imm_sample.head(0))
imm_header = imm_header[1:]
pprint(imm_header)

['cicid',
 'i94yr',
 'i94mon',
 'i94cit',
 'i94res',
 'i94port',
 'arrdate',
 'i94mode',
 'i94addr',
 'depdate',
 'i94bir',
 'i94visa',
 'count',
 'dtadfile',
 'visapost',
 'occup',
 'entdepa',
 'entdepd',
 'entdepu',
 'matflag',
 'biryear',
 'dtaddto',
 'gender',
 'insnum',
 'airline',
 'admnum',
 'fltno',
 'visatype']


In [3]:
# .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0") \
# .config("spark.jars.packages", "saurfang:spark-sas7bdat:3.0.0-s_2.12") \
from pyspark.sql import SparkSession
spark = SparkSession.builder \
                    .enableHiveSupport() \
                    .config("spark.jars.packages", "saurfang:spark-sas7bdat:3.0.0-s_2.12") \
                    .getOrCreate()
# df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [4]:
#write to parquet
# df_spark.write.parquet("sas_data")
# df_spark=spark.read.parquet("sas_data")

# Step 2: Explore and Assess the Data
## Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.
* There're duplicate invalid i94cit, i94res in i94 dataset, also Jun i94 has additional columns compare to others.
* global temperature datasets is from 19th centry to 2013. So I won't use it in this project.
* some airport data is not airport, need to drop those rows.

## Cleaning Steps
Document steps necessary to clean the data

# Performing cleaning tasks here
* Checkout i94 2016 data
    * drop additional columns from Jun 2016, union all to other 11 months into dataframe, drop unused columns (eg. 'visapost', 'occup', 'entdepa')
* Checkout demographics data
    * since demographics data has race/count columns per county, so need to pivot by race, then aggregate to sum/mean columns after group by state.
* Checkout airport data
    * 
* Checkout temperature data

# get valid values of I94 fields

In [4]:
# try configparsing
config = configparser.ConfigParser()
config.read_file(open('meta_i94.cfg'))

pprint(config['I94MODE'])

<Section: I94MODE>


In [5]:
sections_dict = {}
# get sections and iterate over each
sections = config.sections()

for section in sections:
    options = config.options(section)
    temp_dict = {}
    for option in options:
        k = option.strip('\'').strip().upper()
        v = config.get(section,option).strip('\'').strip().upper()
        temp_dict[k] = v

    sections_dict[section] = temp_dict

In [6]:
# cast meta key into it's original type
meta_i94 = {}
for k, v in sections_dict.items():
    print(k)
    if k == 'I94CIT' or k == 'I94MODE' or k == 'I94VISA':
        meta_i94[k] = {int(vk):vv for vk,vv in v.items()}
    else:
        meta_i94[k] = v

I94CIT
I94PORT
I94MODE
I94VISA
I94ADDR


## i94 data

In [55]:
i94_fnames=['./data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat']

In [56]:
CustomI94Schema = R([
    Fld("cicid", Int()),
    Fld("i94yr", Int()),
    Fld("i94mon", Int()),  
    Fld("i94cit", Int()),  
    Fld("i94res", Int()),  
    Fld("i94port", Str()), 
    Fld("arrdate", Int()), 
    Fld("i94mode", Int()), 
    Fld("i94addr", Str()), 
    Fld("depdate", Int()), 
    Fld("i94bir", Int()),  
    Fld("i94visa", Int()), 
    Fld("count", Int()),
    Fld("dtadfile", Str()),
    Fld("visapost", Str()),
    Fld("occup", Str()),
    Fld("entdepa", Str()), 
    Fld("entdepd", Str()), 
    Fld("entdepu", Str()),
    Fld("matflag", Str()),  
    Fld("biryear", Int()), 
    Fld("dtaddto", Str()),
    Fld("gender", Str()),  
    Fld("insnum", Str()),  
    Fld("airline", Str()), 
    Fld("admnum", Dbl()),  
    Fld("fltno", Str()),
    Fld("visatype", Str())
])

CustomI94JunSchema = R([
    Fld("cicid", Int()),
    Fld("i94yr", Int()),
    Fld("i94mon", Int()),  
    Fld("i94cit", Int()),  
    Fld("i94res", Int()),  
    Fld("i94port", Str()), 
    Fld("arrdate", Int()), 
    Fld("i94mode", Int()), 
    Fld("i94addr", Str()), 
    Fld("depdate", Int()), 
    Fld("i94bir", Int()),  
    Fld("i94visa", Int()), 
    Fld("count", Int()),
    Fld("validres", Int()),
    Fld("delete_days", Int()),
    Fld("delete_mexl", Int()),
    Fld("delete_dup", Int()),
    Fld("delete_visa", Int()), 
    Fld("delete_recdup", Int()),
    Fld("dtadfile", Str()),
    Fld("visapost", Str()),
    Fld("occup", Str()),
    Fld("entdepa", Str()), 
    Fld("entdepd", Str()), 
    Fld("entdepu", Str()),
    Fld("matflag", Str()),  
    Fld("biryear", Int()), 
    Fld("dtaddto", Str()),
    Fld("gender", Str()),  
    Fld("insnum", Str()),  
    Fld("airline", Str()), 
    Fld("admnum", Dbl()),  
    Fld("fltno", Str()),
    Fld("visatype", Str())
])

In [58]:
# Examine i94 data
columns_to_drop = ['validres', 'delete_days', 'delete_mexl', 'delete_dup', 'delete_visa', 'delete_recdup']
df_i94s = OrderedDict() 
for f in i94_fnames:
    fname = os.path.basename(f)
    print(fname)
    if "jun" in fname: 
        d = spark.read.schema(CustomI94JunSchema).format('com.github.saurfang.sas.spark').load(f)
        d = d.drop(*columns_to_drop)
    else:
        d = spark.read.schema(CustomI94Schema).format('com.github.saurfang.sas.spark').load(f)
#     d = d.dropna(subset=["cicid", "arrdate", "i94addr"]).dropDuplicates()
    df_i94s[fname] = d
    d.printSchema()
    d.show()

i94_jan16_sub.sas7bdat
root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = tru

In [59]:
# sample 1% of each month of i94, since origin size of sas is around 500MB.
sample_rate = 0.01
df_i94s_sample = {}
for k, v in df_i94s.items():
    df_i94s_sample[k] = v.sample(sample_rate)
    
from functools import reduce
print(len(df_i94s_sample.values()))
df_i94s_sample_unionall = reduce(DataFrame.unionAll, df_i94s_sample.values())

df_i94s_sample_unionall.printSchema()

12
root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: doub

In [60]:
# df_i94s_sample_unionall.write.mode('overwrite').parquet(path='i94_sample.parquet')

In [61]:
%%time
df_sas = spark.read.parquet('sas_data/')
df_sas.count()

CPU times: user 3.32 ms, sys: 0 ns, total: 3.32 ms
Wall time: 439 ms


3096313

In [62]:
%%time
# df_imm = spark.read.parquet('i94_sample.parquet/')
df_imm = df_i94s_sample_unionall.dropna(subset=["cicid", "arrdate", "i94addr"]).dropDuplicates()
df_imm = df_imm.filter( df_imm.i94cit.isin(list(meta_i94['I94CIT'].keys())) )
df_imm = df_imm.filter( df_imm.i94res.isin(list(meta_i94['I94CIT'].keys())) )
# extract valid i94 data
drop_columns = ['i94mode', 'i94bir', 'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu', 'matflag', 'dtaddto', 'insnum', 'admnum', 'fltno']
df_imm = df_imm.drop(*drop_columns)
df_imm.printSchema()
df_imm.show()
# print(df_imm.coun())

root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- visatype: string (nullable = true)

+-------+-----+------+------+------+-------+-------+-------+-------+-------+-------+------+-------+--------+
|  cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94addr|depdate|i94visa|biryear|gender|airline|visatype|
+-------+-----+------+------+------+-------+-------+-------+-------+-------+-------+------+-------+--------+
|   5364| 2016|     1|   113|   113|    NEW|  20468|     PA|  20544|      2|   1964|     M|     LH|      WT|
|  10569| 

In [63]:
# write valid i94 into parquet for Spark
df_imm.write.partitionBy("i94mon", "i94addr").mode('overwrite').parquet(path='i94_valid.parquet')
# df_imm.write.mode('overwrite').parquet(path='i94_valid.parquet')

In [64]:
# examine i94_valid
df_imm = spark.read.parquet('i94_valid.parquet/')
df_imm.createOrReplaceTempView("imm")
imm_table = spark.sql("""
select distinct i94mon, COUNT(*)
FROM imm
GROUP BY 1
ORDER BY 1 asc
""")
imm_table.printSchema()
imm_table.show()
LOG.info(f"total 1% sample parquet rows: {imm_table.count()}")

root
 |-- i94mon: integer (nullable = true)
 |-- count(1): long (nullable = false)

+------+--------+
|i94mon|count(1)|
+------+--------+
|     1|   22896|
|     2|   20691|
|     3|   26046|
|     4|   25957|
|     5|   28551|
|     6|   29919|
|     7|   36362|
|     8|   35194|
|     9|   31308|
|    10|   30747|
|    11|   24475|
|    12|   29461|
+------+--------+



2020-09-30 23:18:01,289 INFO  [etl] total 1% sample parquet rows: 12


In [65]:
test = spark.sql("""
select cicid, count(*)
from imm
group by 1
having count(cicid) > 1
order by 2
""")
test.printSchema()
test.show()
test.count()

root
 |-- cicid: integer (nullable = true)
 |-- count(1): long (nullable = false)

+-------+--------+
|  cicid|count(1)|
+-------+--------+
| 582510|       2|
|1545686|       2|
|4489884|       2|
|2811218|       2|
| 227408|       2|
|3536367|       2|
|  79361|       2|
|5060206|       2|
|1833539|       2|
|2222510|       2|
|3297293|       2|
|1415143|       2|
|2704853|       2|
|2938828|       2|
| 469028|       2|
|5438141|       2|
| 998072|       2|
|4962308|       2|
|2311725|       2|
| 171142|       2|
+-------+--------+
only showing top 20 rows



7580

## demographics data

In [50]:
custom_demo_schema = R([
    Fld("City", Str()),
    Fld("State", Str()),
    Fld("Median_Age", Dbl()),  
    Fld("Male_Population", Int()),
    Fld("Female_Population", Int()),
    Fld("Total_Population", Int()),
    Fld("Number_Veterans", Int()),
    Fld("Foregin_born", Int()),
    Fld("Average_Household", Dbl()),
    Fld("State_Code", Str()),
    Fld("Race", Str()),
    Fld("Count", Int()),
])

df_degh = spark.read.schema(custom_demo_schema).options(header='True',sep=';').csv('./us-cities-demographics.csv')
df_degh.printSchema()
df_degh.show(5)

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median_Age: double (nullable = true)
 |-- Male_Population: integer (nullable = true)
 |-- Female_Population: integer (nullable = true)
 |-- Total_Population: integer (nullable = true)
 |-- Number_Veterans: integer (nullable = true)
 |-- Foregin_born: integer (nullable = true)
 |-- Average_Household: double (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)

+----------------+-------------+----------+---------------+-----------------+----------------+---------------+------------+-----------------+----------+--------------------+-----+
|            City|        State|Median_Age|Male_Population|Female_Population|Total_Population|Number_Veterans|Foregin_born|Average_Household|State_Code|                Race|Count|
+----------------+-------------+----------+---------------+-----------------+----------------+---------------+----

## Checkout airport data

In [192]:
clean_columns = ['ident', 'continent', 'municipality', 'gps_code', 'local_code', 'coord', 'coordinates', 'elevation_ft', 'iso_country']
df_air = spark.read.options(header='True',inferSchema='True').csv('./airport-codes_csv.csv')
df_air = df_air.filter(df_air.iso_country == 'US') \
                .filter(df_air.type.isin(list(["small_airport", "medium_airport", "large_airport"])))
df_air = df_air.dropna(subset=["iata_code"])
df_air = df_air.withColumn("coord", F.split(df_air.coordinates, ",") ) \
                .withColumn("latitude", F.col('coord')[0]) \
                .withColumn("longitude", F.col('coord')[1]) 
df_air = df_air.drop(*clean_columns)

df_air.printSchema()
df_air.show(5)
df_air.count()

root
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)

+-------------+--------------------+----------+---------+------------------+-------------------+
|         type|                name|iso_region|iata_code|          latitude|          longitude|
+-------------+--------------------+----------+---------+------------------+-------------------+
|small_airport|Ocean Reef Club A...|     US-FL|      OCA|  -80.274803161621|    25.325399398804|
|small_airport|Pilot Station Air...|     US-AK|      PQS|       -162.899994|          61.934601|
|small_airport|Crested Butte Air...|     US-CO|      CSE|       -106.928341|          38.851918|
|small_airport|   LBJ Ranch Airport|     US-TX|      JCY|-98.62249755859999| 30.251800537100003|
|small_airport|Metropolitan Airport|     US-MA|      PMX|-72.31140136719999| 42

1865

In [193]:
df_air.write.mode('overwrite').parquet(path='df_airport.parquet')

## Checkout temperature data

In [236]:
df_tmpt = spark.read.options(header='True', inferSchema='True').csv('./GlobalLandTemperaturesByCity.csv')
df_tmpt = df_tmpt.filter(df_tmpt.Country == "United States")
df_tmpt.printSchema()
df_tmpt.show(5)

root
 |-- dt: string (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)

+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|   City|      Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|1820-01-01|2.1010000000000004|                        3.217|Abilene|United States|  32.95N|  100.53W|
|1820-02-01|             6.926|                        2.853|Abilene|United States|  32.95N|  100.53W|
|1820-03-01|            10.767|                        2.395|Abilene|United States|  32.95N|  100.53W|
|1820-04-01|17.988999999999994|                        2.202|Abilene|United

In [237]:
df_tmpt.write.mode('overwrite').parquet(path='temperature_us.parquet')

In [258]:
df_temper = spark.read.parquet('./temperature_us.parquet/')
df_temper = df_temper.withColumn("datetime", F.col("dt").cast("date")).drop("dt")
df_temper = df_temper.filter( df_temper["datetime"] >= F.lit('2016-01-01') )
df_temper.printSchema()
df_temper.show()

root
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- datetime: date (nullable = true)

+------------------+-----------------------------+----+-------+--------+---------+--------+
|AverageTemperature|AverageTemperatureUncertainty|City|Country|Latitude|Longitude|datetime|
+------------------+-----------------------------+----+-------+--------+---------+--------+
+------------------+-----------------------------+----+-------+--------+---------+--------+



# Step 3: Define the Data Model
## 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model
### dimension table:
1. df_i94visa: visa_id (PK), visa_type.
2. df_i94port: port_id (PK), city, state.
3. df_air: type, name, iso_region, iata_code (PK), latitude, longitude.
4. df_i94cit: citres_id (PK), citres.
5. df_i94addr: addr_id (PK), state.
6. df_demo: State_Code(PK), Median_Age, Male_Population, Female_Population, Total_Population, Number_Veterans, Foregin_born, Average_Household, American_Indian_Alaska_Native, Asian, Black_African_American, Hispanic_Latino, White

#### fact table
* df_fact: fact_id (PK), cicid, visatype, i94port (FK), state, name, i94cit (FK), i94addr (FK), total_Population, foregin_born, American_Indian_Alaska_Native, Asian, Black_African_American, Hispanic_Latino, White

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model
![](star.png)

## Extract i94 table from dataframe

In [67]:
%%time
df_i94 = spark.read.parquet('i94_valid.parquet/')
# df_i94 = df_i94.filter( df_i94.i94cit.isin(set(meta_i94['I94CIT'].keys())) )
# df_i94 = df_i94.filter( df_i94.i94res.isin(set(meta_i94['I94CIT'].keys())) )
df_i94.printSchema()
df_i94.show()

root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- visatype: string (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94addr: string (nullable = true)

+-------+-----+------+------+-------+-------+-------+-------+-------+------+-------+--------+------+-------+
|  cicid|i94yr|i94cit|i94res|i94port|arrdate|depdate|i94visa|biryear|gender|airline|visatype|i94mon|i94addr|
+-------+-----+------+------+-------+-------+-------+-------+-------+------+-------+--------+------+-------+
| 113995| 2016|   691|   691|    MIA|  20789|  20797|      2|   1981|     M|     AV|      B2|    12|     FL|
| 133055| 

# Create i94 staging table

In [68]:
df_i94.createOrReplaceTempView("i94")
i94_stage = spark.sql("""
SELECT 
    distinct cicid as cid,
    i94yr,
    i94mon,
    EXTRACT(DAY FROM date_add('1960-01-01',arrdate) ) as i94day,
    i94cit,
    i94res,
    i94port,
    i94addr,
    i94visa,
    biryear,
    gender,
    airline,
    visatype,
    date_add('1960-01-01',arrdate) as arrdate,
    date_add('1960-01-01',depdate) as depdate
FROM i94
order by i94cit asc
""")
i94_stage.printSchema()
i94_stage.show()

root
 |-- cid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94day: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- visatype: string (nullable = true)
 |-- arrdate: date (nullable = true)
 |-- depdate: date (nullable = true)

+-------+-----+------+------+------+------+-------+-------+-------+-------+------+-------+--------+----------+----------+
|    cid|i94yr|i94mon|i94day|i94cit|i94res|i94port|i94addr|i94visa|biryear|gender|airline|visatype|   arrdate|   depdate|
+-------+-----+------+------+------+------+-------+-------+-------+-------+------+-------+--------+----------+----------+
|1659514| 2016|     7|     8|   101|   101|    NE

# Create airport dim table

In [21]:
meta_i94.keys()

dict_keys(['I94CIT', 'I94PORT', 'I94MODE', 'I94VISA', 'I94ADDR'])

In [23]:
i94port = meta_i94['I94PORT']
pddf = pd.DataFrame(i94port.items(), columns=["id", "airport"])
df_i94port = spark.createDataFrame(pddf)
df_i94port.printSchema()
df_i94port.show()

root
 |-- id: string (nullable = true)
 |-- airport: string (nullable = true)

+---+--------------------+
| id|             airport|
+---+--------------------+
|ALC|           ALCAN, AK|
|ANC|       ANCHORAGE, AK|
|BAR|BAKER AAF - BAKER...|
|DAC|   DALTONS CACHE, AK|
|PIZ|DEW STATION PT LA...|
|DTH|    DUTCH HARBOR, AK|
|EGL|           EAGLE, AK|
|FRB|       FAIRBANKS, AK|
|HOM|           HOMER, AK|
|HYD|           HYDER, AK|
|JUN|          JUNEAU, AK|
|5KE|       KETCHIKAN, AK|
|KET|       KETCHIKAN, AK|
|MOS|MOSES POINT INTER...|
|NIK|         NIKISKI, AK|
|NOM|             NOM, AK|
|PKC|     POKER CREEK, AK|
|ORI|  PORT LIONS SPB, AK|
|SKA|         SKAGWAY, AK|
|SNP| ST. PAUL ISLAND, AK|
+---+--------------------+
only showing top 20 rows



In [118]:
meta_i94['I94ADDR'].keys()

dict_keys(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'GU', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NC', 'ND', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VI', 'VA', 'WV', 'WA', 'WI', 'WY', '99'])

In [154]:
i94port = meta_i94['I94PORT']
pddf = pd.DataFrame(i94port.items(), columns=["port_id", "location"])
df_i94port = spark.createDataFrame(pddf)
df_i94port = df_i94port.withColumn("loc", F.split(F.col('location'), ",") ) \
                .withColumn("city", F.trim( F.col('loc')[0]) ) \
                .withColumn("state", F.trim( F.col('loc')[1]) ) \
                .drop("loc").drop('location')
print(df_i94port.count())
df_i94port.printSchema()
df_i94port.show()

588
root
 |-- port_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)

+-------+--------------------+-----+
|port_id|                city|state|
+-------+--------------------+-----+
|    ALC|               ALCAN|   AK|
|    ANC|           ANCHORAGE|   AK|
|    BAR|BAKER AAF - BAKER...|   AK|
|    DAC|       DALTONS CACHE|   AK|
|    PIZ|DEW STATION PT LA...|   AK|
|    DTH|        DUTCH HARBOR|   AK|
|    EGL|               EAGLE|   AK|
|    FRB|           FAIRBANKS|   AK|
|    HOM|               HOMER|   AK|
|    HYD|               HYDER|   AK|
|    JUN|              JUNEAU|   AK|
|    5KE|           KETCHIKAN|   AK|
|    KET|           KETCHIKAN|   AK|
|    MOS|MOSES POINT INTER...|   AK|
|    NIK|             NIKISKI|   AK|
|    NOM|                 NOM|   AK|
|    PKC|         POKER CREEK|   AK|
|    ORI|      PORT LIONS SPB|   AK|
|    SKA|             SKAGWAY|   AK|
|    SNP|     ST. PAUL ISLAND|   AK|
+-------+------------------

In [155]:
df_i94port = df_i94port.filter( F.col('state').isin(list(meta_i94['I94ADDR'].keys())) ) #  F.length(F.col('state')) == 2 
df_i94port.printSchema()
df_i94port.show()

root
 |-- port_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)

+-------+--------------------+-----+
|port_id|                city|state|
+-------+--------------------+-----+
|    ALC|               ALCAN|   AK|
|    ANC|           ANCHORAGE|   AK|
|    BAR|BAKER AAF - BAKER...|   AK|
|    DAC|       DALTONS CACHE|   AK|
|    PIZ|DEW STATION PT LA...|   AK|
|    DTH|        DUTCH HARBOR|   AK|
|    EGL|               EAGLE|   AK|
|    FRB|           FAIRBANKS|   AK|
|    HOM|               HOMER|   AK|
|    HYD|               HYDER|   AK|
|    JUN|              JUNEAU|   AK|
|    5KE|           KETCHIKAN|   AK|
|    KET|           KETCHIKAN|   AK|
|    MOS|MOSES POINT INTER...|   AK|
|    NIK|             NIKISKI|   AK|
|    NOM|                 NOM|   AK|
|    PKC|         POKER CREEK|   AK|
|    ORI|      PORT LIONS SPB|   AK|
|    SKA|             SKAGWAY|   AK|
|    SNP|     ST. PAUL ISLAND|   AK|
+-------+--------------------+-

In [156]:
df_i94port.write.mode('overwrite').parquet(path='df_i94port.parquet')

# Create Visa table for 1,2,3

In [41]:
visa = meta_i94['I94VISA']
print(visa)

{1: 'BUSINESS', 2: 'PLEASURE', 3: 'STUDENT'}


In [42]:
pddf = pd.DataFrame(visa.items(), columns=["visa_id", "visa_type"])
df_i94visa = spark.createDataFrame(pddf)
df_i94visa.printSchema()
df_i94visa.show()

root
 |-- visa_id: long (nullable = true)
 |-- visa_type: string (nullable = true)

+-------+---------+
|visa_id|visa_type|
+-------+---------+
|      1| BUSINESS|
|      2| PLEASURE|
|      3|  STUDENT|
+-------+---------+



In [43]:
df_i94visa.write.mode('overwrite').parquet(path='df_i94visa.parquet')

# Create cit_res dim table

In [44]:
cit = meta_i94['I94CIT']
pddf = pd.DataFrame(cit.items(), columns=["citres_id", "citres"])
df_i94cit = spark.createDataFrame(pddf)
df_i94cit.printSchema()
df_i94cit.show()

root
 |-- citres_id: long (nullable = true)
 |-- citres: string (nullable = true)

+---------+--------------------+
|citres_id|              citres|
+---------+--------------------+
|      582|MEXICO AIR SEA, A...|
|      236|         AFGHANISTAN|
|      101|             ALBANIA|
|      316|             ALGERIA|
|      102|             ANDORRA|
|      324|              ANGOLA|
|      529|            ANGUILLA|
|      518|     ANTIGUA-BARBUDA|
|      687|           ARGENTINA|
|      151|             ARMENIA|
|      532|               ARUBA|
|      438|           AUSTRALIA|
|      103|             AUSTRIA|
|      152|          AZERBAIJAN|
|      512|             BAHAMAS|
|      298|             BAHRAIN|
|      274|          BANGLADESH|
|      513|            BARBADOS|
|      104|             BELGIUM|
|      581|              BELIZE|
+---------+--------------------+
only showing top 20 rows



In [45]:
df_i94cit.write.mode('overwrite').parquet(path='df_i94cit.parquet')

# Create addr dim table

In [47]:
meta_i94.keys()
addr = meta_i94['I94ADDR']
pddf = pd.DataFrame(addr.items(), columns=["addr_id", "state"])
df_i94addr = spark.createDataFrame(pddf)
df_i94addr.printSchema()
df_i94addr.show()

da_1 = df_i94addr.filter( df_i94addr.addr_id == 'CA')
da_1.show()

root
 |-- addr_id: string (nullable = true)
 |-- state: string (nullable = true)

+-------+-----------------+
|addr_id|            state|
+-------+-----------------+
|     AL|          ALABAMA|
|     AK|           ALASKA|
|     AZ|          ARIZONA|
|     AR|         ARKANSAS|
|     CA|       CALIFORNIA|
|     CO|         COLORADO|
|     CT|      CONNECTICUT|
|     DE|         DELAWARE|
|     DC|DIST. OF COLUMBIA|
|     FL|          FLORIDA|
|     GA|          GEORGIA|
|     GU|             GUAM|
|     HI|           HAWAII|
|     ID|            IDAHO|
|     IL|         ILLINOIS|
|     IN|          INDIANA|
|     IA|             IOWA|
|     KS|           KANSAS|
|     KY|         KENTUCKY|
|     LA|        LOUISIANA|
+-------+-----------------+
only showing top 20 rows

+-------+----------+
|addr_id|     state|
+-------+----------+
|     CA|CALIFORNIA|
+-------+----------+



In [48]:
df_i94addr.write.mode('overwrite').parquet(path='df_i94addr.parquet')

# Create demopgrahics dim table

In [86]:
# df_demo_1 = df_degh.filter(df_degh["State_Code"] == "ID")
df_demo_1 = df_degh
df_demo_1.printSchema()
df_demo_1.show()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median_Age: double (nullable = true)
 |-- Male_Population: integer (nullable = true)
 |-- Female_Population: integer (nullable = true)
 |-- Total_Population: integer (nullable = true)
 |-- Number_Veterans: integer (nullable = true)
 |-- Foregin_born: integer (nullable = true)
 |-- Average_Household: double (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)

+----------------+--------------+----------+---------------+-----------------+----------------+---------------+------------+-----------------+----------+--------------------+------+
|            City|         State|Median_Age|Male_Population|Female_Population|Total_Population|Number_Veterans|Foregin_born|Average_Household|State_Code|                Race| Count|
+----------------+--------------+----------+---------------+-----------------+----------------+---------------

In [87]:
# https://databricks.com/blog/2016/02/09/reshaping-data-with-pivot-in-apache-spark.html
df_pi = df_demo_1.groupby("State_Code","Median_Age","Male_Population","Female_Population","Total_Population","Number_Veterans","Foregin_born","Average_Household").pivot("Race").sum("Count")
df_pi.printSchema()
df_pi.show()

root
 |-- State_Code: string (nullable = true)
 |-- Median_Age: double (nullable = true)
 |-- Male_Population: integer (nullable = true)
 |-- Female_Population: integer (nullable = true)
 |-- Total_Population: integer (nullable = true)
 |-- Number_Veterans: integer (nullable = true)
 |-- Foregin_born: integer (nullable = true)
 |-- Average_Household: double (nullable = true)
 |-- American Indian and Alaska Native: long (nullable = true)
 |-- Asian: long (nullable = true)
 |-- Black or African-American: long (nullable = true)
 |-- Hispanic or Latino: long (nullable = true)
 |-- White: long (nullable = true)

+----------+----------+---------------+-----------------+----------------+---------------+------------+-----------------+---------------------------------+-----+-------------------------+------------------+------+
|State_Code|Median_Age|Male_Population|Female_Population|Total_Population|Number_Veterans|Foregin_born|Average_Household|American Indian and Alaska Native|Asian|Black or A

In [88]:
df_demo = df_pi.groupby('State_Code').agg(F.avg('Median_Age').alias('Median_Age'), \
                                         F.sum('Male_Population').alias('Male_Population'), \
                                         F.sum('Female_Population').alias('Female_Population'), \
                                         F.sum('Total_Population').alias('Total_Population'), \
                                         F.sum('Number_Veterans').alias('Number_Veterans'), 
                                         F.sum('Foregin_born').alias('Foregin_born'), \
                                         F.avg('Average_Household').alias('Average_Household'), \
                                         F.sum('American Indian and Alaska Native').alias('American_Indian_Alaska_Native'), \
                                         F.sum('Asian').alias('Asian'), \
                                         F.sum('Black or African-American').alias('Black_African_American'), \
                                         F.sum('Hispanic or Latino').alias('Hispanic_Latino'), \
                                         F.sum('White').alias('White') \
                                        )
df_demo.printSchema()
df_demo.show()

root
 |-- State_Code: string (nullable = true)
 |-- Median_Age: double (nullable = true)
 |-- Male_Population: long (nullable = true)
 |-- Female_Population: long (nullable = true)
 |-- Total_Population: long (nullable = true)
 |-- Number_Veterans: long (nullable = true)
 |-- Foregin_born: long (nullable = true)
 |-- Average_Household: double (nullable = true)
 |-- American_Indian_Alaska_Native: long (nullable = true)
 |-- Asian: long (nullable = true)
 |-- Black_African_American: long (nullable = true)
 |-- Hispanic_Latino: long (nullable = true)
 |-- White: long (nullable = true)

+----------+------------------+---------------+-----------------+----------------+---------------+------------+------------------+-----------------------------+-------+----------------------+---------------+--------+
|State_Code|        Median_Age|Male_Population|Female_Population|Total_Population|Number_Veterans|Foregin_born| Average_Household|American_Indian_Alaska_Native|  Asian|Black_African_American|Hi

In [89]:
df_demo.write.mode('overwrite').parquet(path='df_demo.parquet')

# Step 4: Run Pipelines to Model the Data 
## 4.1 Create the data model
Build the data pipelines to create the data model.

# Build fact table

In [157]:
# Read dim parquet
df_i94visa = spark.read.parquet('./df_i94visa.parquet/')
df_i94port = spark.read.parquet('./df_i94port.parquet/')
df_air = spark.read.parquet('./df_airport.parquet/')
df_i94cit = spark.read.parquet('./df_i94cit.parquet/')
df_i94addr = spark.read.parquet('./df_i94addr.parquet/')
df_demo = spark.read.parquet('./df_demo.parquet/')
df_imm = spark.read.parquet('./i94_valid.parquet/')

In [158]:
df_i94visa.createOrReplaceTempView('i94visa')
df_i94port.createOrReplaceTempView('i94port')
df_air.createOrReplaceTempView('air')
df_i94cit.createOrReplaceTempView('i94cit')
df_i94addr.createOrReplaceTempView('i94addr')
df_demo.createOrReplaceTempView('demo')
df_imm.createOrReplaceTempView('i94')

In [164]:
df_imm.printSchema()
df_imm.show(3)

root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- visatype: string (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94addr: string (nullable = true)

+------+-----+------+------+-------+-------+-------+-------+-------+------+-------+--------+------+-------+
| cicid|i94yr|i94cit|i94res|i94port|arrdate|depdate|i94visa|biryear|gender|airline|visatype|i94mon|i94addr|
+------+-----+------+------+-------+-------+-------+-------+-------+------+-------+--------+------+-------+
|113995| 2016|   691|   691|    MIA|  20789|  20797|      2|   1981|     M|     AV|      B2|    12|     FL|
|133055| 2016|

## create fact immigration table

In [168]:
# extract columns from joined tables to create fact table
from pyspark.sql.functions import monotonically_increasing_id
df_fact = spark.sql("""
SELECT
    monotonically_increasing_id() as fact_id,
    cicid,
    visatype,
    i94port,
    p.state,
    air.name,
    i94cit,
    c.citres
    i94addr,
    d.Total_Population,
    d.Foregin_born,
    d.American_Indian_Alaska_Native,
    d.Asian,
    d.Black_African_American,
    d.Hispanic_Latino,
    d.White
FROM i94
LEFT JOIN i94visa as v on i94.i94visa = v.visa_id
LEFT JOIN i94port as p on i94.i94port = p.port_id
LEFT JOIN air on i94.i94port = air.iata_code
LEFT JOIN i94cit as c on i94.i94cit = c.citres_id
LEFT JOIN i94addr as a on i94.i94addr = a.addr_id
LEFT JOIN demo as d on i94.i94addr = d.State_Code
""")

df_fact.printSchema()
df_fact.show()

root
 |-- fact_id: long (nullable = false)
 |-- cicid: integer (nullable = true)
 |-- visatype: string (nullable = true)
 |-- i94port: string (nullable = true)
 |-- state: string (nullable = true)
 |-- name: string (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- Total_Population: long (nullable = true)
 |-- Foregin_born: long (nullable = true)
 |-- American_Indian_Alaska_Native: long (nullable = true)
 |-- Asian: long (nullable = true)
 |-- Black_African_American: long (nullable = true)
 |-- Hispanic_Latino: long (nullable = true)
 |-- White: long (nullable = true)

+-------+-------+--------+-------+-----+--------------------+------+--------------------+----------------+------------+-----------------------------+------+----------------------+---------------+-------+
|fact_id|  cicid|visatype|i94port|state|                name|i94cit|             i94addr|Total_Population|Foregin_born|American_Indian_Alaska_Native| Asian|Black_African_

In [166]:
df_fact.count()

341607

In [169]:
# write fact table into parquet
df_fact.write.partitionBy("visatype", "state").mode('overwrite').parquet(path='df_fact.parquet')

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

# Check source/count for completeness of dimention tables and fact table

In [170]:
# Perform quality checks here
def not_empty(df, df_name):
    print(f"checking if {df_name} quality, not empty.")
    row_cnt = df.count()
    print(f"{df_name}'s size: {row_cnt}")
    print(f"{df_name}'s schema, sample rows")
    df.printSchema()
    df.show()
    
## check visa table is not empty 
not_empty(df_i94visa, "df_i94visa")
not_empty(df_i94port, "df_i94port")
not_empty(df_i94cit, "df_i94cit")
not_empty(df_demo, "df_demo")
not_empty(df_fact, "df_fact")

checking if df_i94visa quality, not empty.
df_i94visa's size: 3
df_i94visa's schema, sample rows
root
 |-- visa_id: long (nullable = true)
 |-- visa_type: string (nullable = true)

+-------+---------+
|visa_id|visa_type|
+-------+---------+
|      2| PLEASURE|
|      1| BUSINESS|
|      3|  STUDENT|
+-------+---------+

checking if df_i94port quality, not empty.
df_i94port's size: 500
df_i94port's schema, sample rows
root
 |-- port_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)

+-------+--------------------+-----+
|port_id|                city|state|
+-------+--------------------+-----+
|    PER|         PERTH AMBOY|   NJ|
|    ACY|POMONA FIELD - AT...|   NJ|
|    ABQ|         ALBUQUERQUE|   NM|
|    ANP|      ANTELOPE WELLS|   NM|
|    CRL|            CARLSBAD|   NM|
|    COL|            COLUMBUS|   NM|
|    CDD|CRANE LAKE - ST. ...|   NM|
|    LAS|          LAS CRUCES|   NM|
|    RUI|             RUIDOSO|   NM|
|    STR|        

# Check dimension table has valid, range of date

In [171]:
# Check fact table has same state as in demographics, airports
# FROM i94
# LEFT JOIN i94visa as v on i94.i94visa = v.visa_id
# LEFT JOIN i94port as p on i94.i94port = p.port_id
# LEFT JOIN air on i94.i94port = air.iata_code
# LEFT JOIN i94cit as c on i94.i94cit = c.citres_id
# LEFT JOIN i94addr as a on i94.i94addr = a.addr_id
# LEFT JOIN demo as d on i94.i94addr = d.State_Code
## demo join port
demo_airport_tbl = spark.sql("""
SELECT 
    state, count(*)
FROM i94port 
GROUP BY 1
ORDER BY 2 desc
""")
not_empty(demo_airport_tbl, "demo_airport_tbl")
## demo join addr

checking if demo_airport_tbl quality, not empty.
demo_airport_tbl's size: 53
demo_airport_tbl's schema, sample rows
root
 |-- state: string (nullable = true)
 |-- count(1): long (nullable = false)

+-----+--------+
|state|count(1)|
+-----+--------+
|   TX|      47|
|   NY|      34|
|   WA|      30|
|   FL|      29|
|   MT|      26|
|   ME|      26|
|   CA|      25|
|   MN|      22|
|   AK|      22|
|   ND|      21|
|   MI|      16|
|   VT|      16|
|   PR|      12|
|   OH|      11|
|   AZ|      11|
|   NJ|       9|
|   NC|       8|
|   NM|       8|
|   CT|       7|
|   IL|       7|
+-----+--------+
only showing top 20 rows



#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

In [189]:
df_i94visa.printSchema()
df_i94visa.show()

root
 |-- visa_id: long (nullable = true)
 |-- visa_type: string (nullable = true)

+-------+---------+
|visa_id|visa_type|
+-------+---------+
|      2| PLEASURE|
|      1| BUSINESS|
|      3|  STUDENT|
+-------+---------+



In [190]:
df_i94port.printSchema()
df_i94port.show()

root
 |-- port_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)

+-------+--------------------+-----+
|port_id|                city|state|
+-------+--------------------+-----+
|    PER|         PERTH AMBOY|   NJ|
|    ACY|POMONA FIELD - AT...|   NJ|
|    ABQ|         ALBUQUERQUE|   NM|
|    ANP|      ANTELOPE WELLS|   NM|
|    CRL|            CARLSBAD|   NM|
|    COL|            COLUMBUS|   NM|
|    CDD|CRANE LAKE - ST. ...|   NM|
|    LAS|          LAS CRUCES|   NM|
|    RUI|             RUIDOSO|   NM|
|    STR|        SANTA TERESA|   NM|
|    RNO|CANNON INTL - REN...|   NV|
|    FLX|FALLON MUNICIPAL ...|   NV|
|    LVG|           LAS VEGAS|   NV|
|    REN|                RENO|   NV|
|    ALB|              ALBANY|   NY|
|    AXB|      ALEXANDRIA BAY|   NY|
|    BUF|             BUFFALO|   NY|
|    CNH|      CANNON CORNERS|   NY|
|    CAP|        CAPE VINCENT|   NY|
|    CHM|           CHAMPLAIN|   NY|
+-------+--------------------+-

In [194]:
df_air.printSchema()
df_air.show()

root
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)

+-------------+--------------------+----------+---------+------------------+-------------------+
|         type|                name|iso_region|iata_code|          latitude|          longitude|
+-------------+--------------------+----------+---------+------------------+-------------------+
|small_airport|Ocean Reef Club A...|     US-FL|      OCA|  -80.274803161621|    25.325399398804|
|small_airport|Pilot Station Air...|     US-AK|      PQS|       -162.899994|          61.934601|
|small_airport|Crested Butte Air...|     US-CO|      CSE|       -106.928341|          38.851918|
|small_airport|   LBJ Ranch Airport|     US-TX|      JCY|-98.62249755859999| 30.251800537100003|
|small_airport|Metropolitan Airport|     US-MA|      PMX|-72.31140136719999| 42

In [195]:
df_i94cit.printSchema()
df_i94cit.show()

root
 |-- citres_id: long (nullable = true)
 |-- citres: string (nullable = true)

+---------+--------------------+
|citres_id|              citres|
+---------+--------------------+
|      582|MEXICO AIR SEA, A...|
|      236|         AFGHANISTAN|
|      101|             ALBANIA|
|      316|             ALGERIA|
|      102|             ANDORRA|
|      324|              ANGOLA|
|      529|            ANGUILLA|
|      518|     ANTIGUA-BARBUDA|
|      687|           ARGENTINA|
|      151|             ARMENIA|
|      532|               ARUBA|
|      438|           AUSTRALIA|
|      103|             AUSTRIA|
|      152|          AZERBAIJAN|
|      512|             BAHAMAS|
|      298|             BAHRAIN|
|      274|          BANGLADESH|
|      513|            BARBADOS|
|      104|             BELGIUM|
|      581|              BELIZE|
+---------+--------------------+
only showing top 20 rows



In [177]:
df_i94addr.printSchema()
df_i94addr.show()

root
 |-- addr_id: string (nullable = true)
 |-- state: string (nullable = true)

+-------+---------------+
|addr_id|          state|
+-------+---------------+
|     PA|   PENNSYLVANIA|
|     PR|    PUERTO RICO|
|     RI|   RHODE ISLAND|
|     SC|    S. CAROLINA|
|     SD|      S. DAKOTA|
|     TN|      TENNESSEE|
|     TX|          TEXAS|
|     UT|           UTAH|
|     VT|        VERMONT|
|     VI| VIRGIN ISLANDS|
|     VA|       VIRGINIA|
|     WV|    W. VIRGINIA|
|     WA|     WASHINGTON|
|     WI|      WISCONSON|
|     WY|        WYOMING|
|     99|ALL OTHER CODES|
|     ID|          IDAHO|
|     IL|       ILLINOIS|
|     IN|        INDIANA|
|     IA|           IOWA|
+-------+---------------+
only showing top 20 rows



In [178]:
df_demo.printSchema()
df_demo.show()

root
 |-- State_Code: string (nullable = true)
 |-- Median_Age: double (nullable = true)
 |-- Male_Population: long (nullable = true)
 |-- Female_Population: long (nullable = true)
 |-- Total_Population: long (nullable = true)
 |-- Number_Veterans: long (nullable = true)
 |-- Foregin_born: long (nullable = true)
 |-- Average_Household: double (nullable = true)
 |-- American_Indian_Alaska_Native: long (nullable = true)
 |-- Asian: long (nullable = true)
 |-- Black_African_American: long (nullable = true)
 |-- Hispanic_Latino: long (nullable = true)
 |-- White: long (nullable = true)

+----------+------------------+---------------+-----------------+----------------+---------------+------------+------------------+-----------------------------+------+----------------------+---------------+--------+
|State_Code|        Median_Age|Male_Population|Female_Population|Total_Population|Number_Veterans|Foregin_born| Average_Household|American_Indian_Alaska_Native| Asian|Black_African_American|Hisp

In [179]:
df_fact.printSchema()
df_fact.show()

root
 |-- fact_id: long (nullable = false)
 |-- cicid: integer (nullable = true)
 |-- visatype: string (nullable = true)
 |-- i94port: string (nullable = true)
 |-- state: string (nullable = true)
 |-- name: string (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- Total_Population: long (nullable = true)
 |-- Foregin_born: long (nullable = true)
 |-- American_Indian_Alaska_Native: long (nullable = true)
 |-- Asian: long (nullable = true)
 |-- Black_African_American: long (nullable = true)
 |-- Hispanic_Latino: long (nullable = true)
 |-- White: long (nullable = true)

+-------+-------+--------+-------+-----+--------------------+------+--------------------+----------------+------------+-----------------------------+------+----------------------+---------------+-------+
|fact_id|  cicid|visatype|i94port|state|                name|i94cit|             i94addr|Total_Population|Foregin_born|American_Indian_Alaska_Native| Asian|Black_African_

In [180]:
# Find top states that most Students/Worker visit

In [182]:
df_fact.select('visatype').distinct().collect()

[Row(visatype='F2'),
 Row(visatype='GMB'),
 Row(visatype='B2'),
 Row(visatype='F1'),
 Row(visatype='I1'),
 Row(visatype='WB'),
 Row(visatype='M1'),
 Row(visatype='B1'),
 Row(visatype='WT'),
 Row(visatype='M2'),
 Row(visatype='CP'),
 Row(visatype='GMT'),
 Row(visatype='E1'),
 Row(visatype='I'),
 Row(visatype='E2')]

In [184]:
df_fact.createOrReplaceTempView("fact_tbl")
top_states_fo_visa = spark.sql("""
select 
    visatype, COUNT(visatype)
from fact_tbl
group by 1
order by 2 desc
""")
top_states_fo_visa.printSchema()
top_states_fo_visa.show()

root
 |-- visatype: string (nullable = true)
 |-- count(visatype): long (nullable = false)

+--------+---------------+
|visatype|count(visatype)|
+--------+---------------+
|      B2|         140047|
|      WT|         136919|
|      WB|          23439|
|      B1|          20529|
|      F1|          12950|
|     GMT|           3061|
|      E2|           2013|
|      CP|           1299|
|      F2|            539|
|      E1|            327|
|       I|            316|
|      M1|            125|
|      I1|             29|
|     GMB|              8|
|      M2|              6|
+--------+---------------+



In [188]:
top_states_f1_visa = spark.sql("""
select 
    visatype, state, COUNT(visatype)
from fact_tbl
where visatype == 'F1'
group by 1,2
order by 3 desc
""")
top_states_f1_visa.printSchema()
top_states_f1_visa.show()

root
 |-- visatype: string (nullable = true)
 |-- state: string (nullable = true)
 |-- count(visatype): long (nullable = false)

+--------+-----+---------------+
|visatype|state|count(visatype)|
+--------+-----+---------------+
|      F1|   CA|           2956|
|      F1|   NY|           2321|
|      F1|   IL|           1494|
|      F1|   TX|           1092|
|      F1|   FL|            934|
|      F1|   MA|            816|
|      F1| null|            772|
|      F1|   NJ|            632|
|      F1|   WA|            538|
|      F1|   GA|            386|
|      F1|   MI|            339|
|      F1|   PA|            170|
|      F1|   NC|             95|
|      F1|   MN|             81|
|      F1|   AZ|             64|
|      F1|   HI|             61|
|      F1|   CO|             52|
|      F1|   OR|             28|
|      F1|   MD|             22|
|      F1|   PR|             21|
+--------+-----+---------------+
only showing top 20 rows



# Step 5: Complete Project Write Up
## Clearly state the rationale for the choice of tools and technologies for the project.
* In this project, I'm using AWS EMR & Apache Spark to create data lake from i94, airport, us-cities, and demographics datasets by building Star schema with dimension and fact tables.
* The reason to pick AWS EMR is that it's super easy to horizontal and vertical scale by selecting better instance type, and scale to more nodes. Given the immigration data is 60GB for 1 year. It requires scalable solution. Given data lake solution, it's easy to meet ad-doc query from different data users to achieve their business decisions.
* Since the data is not alwways update, it's easy to store data in S3 to persist with low cost.

## Propose how often the data should be updated and why.
* Since we got immigration everyday, this dataset will be update per day. It'll be more efficient to have step function or airflow pipeline solution to do data lineage/clean/ELT. 
* So as to get dimension tables update, and persist in parquet, and cp to S3 for persist, and run ELT with spark to update fact table. Therefore end users can get updated results from new queries.
    
## Write a description of how you would approach the problem differently under the following scenarios:
* The data was increased by 100x.
 * Since S3 is easy to scale and unlimited space, there's no limit on data persist.
 * To catup processing speed, we can do stream processing which is supported by Spark.
 * To handle peek data ingestion, we can scale EMR on-demand to handle.

* The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * Use AWS Step functions or Airflow pipeline to do schedule/cron jobs.

* The database needed to be accessed by 100+ people.
 * Given the fact table is stored in S3, 100x end user should not encounter throughput bottleneck.